## Notebook for Session 9

**Note:** This notebook is to be seen as a 'scratch-pad', illustrating the points made in the slides.

In [1]:
import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
#import tensorflow_hub as hub
from time import time

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Lambda, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.backend import sparse_categorical_crossentropy

### 1. Generation of Some Synthetic Test Data 

Imagine 400 train and 400 test examples with 10 features each. Each example has **two** labels, label 'a' and label 'b'. We construct these examples synthetically by picking 10 random numbers between 0 and 1, and adding for the examples with label $a =  1$ 0.2 to the first five features, and subtracting 0.2 in those features for examples with label $a = 0$. We do the same for label $b$, where we add/subtract 0.1 to features 6-10. (Obviously, this is just a toy example and the details do not matter - we just want to have some data to explore the formalisms.)

In [2]:
train_1_X = np.array([0.2] * 5 + [0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
train_2_X = np.array([0.2] * 5 + [-0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
train_3_X = np.array([-0.2] * 5 + [0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
train_4_X = np.array([-0.2] * 5 + [-0.1] * 5) + np.reshape(np.random.random(1000),[100,10])

train_X = np.concatenate([train_1_X, train_2_X, train_3_X, train_4_X])

test_1_X = np.array([0.2] * 5 + [0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
test_2_X = np.array([0.2] * 5 + [-0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
test_3_X = np.array([-0.2] * 5 + [0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
test_4_X = np.array([-0.2] * 5 + [-0.1] * 5) + np.reshape(np.random.random(1000),[100,10])

test_X = np.concatenate([test_1_X, test_2_X, test_3_X, test_4_X])

train_1a_y = train_1b_y = train_2a_y = train_3b_y = np.reshape([[1] * 100], [100,1])
train_2b_y = train_3a_y = train_4a_y = train_4b_y = np.reshape([[0] * 100], [100,1])

test_1a_y = test_1b_y = test_2a_y = test_3b_y = np.reshape([[1] * 100], [100,1])
test_2b_y = test_3a_y = test_4a_y = test_4b_y = np.reshape([[0] * 100], [100,1])

train_a_y = np.concatenate([train_1a_y, train_2a_y, train_3a_y, train_4a_y])
train_b_y = np.concatenate([train_1b_y, train_2b_y, train_3b_y, train_4b_y])

test_a_y = np.concatenate([test_1a_y, test_2a_y, test_3a_y, test_4a_y])
test_b_y = np.concatenate([test_1b_y, test_2b_y, test_3b_y, test_4b_y])


Shapes as expected?

In [3]:
np.shape(train_X)

(400, 10)

In [4]:
train_X[:2,:]

array([[1.14880381, 0.20888471, 0.37593762, 0.688705  , 0.34861701,
        0.84755726, 0.47453406, 0.10178324, 0.48226536, 0.30355211],
       [0.62609638, 0.59073858, 0.80794111, 0.39427203, 1.10313945,
        0.71879424, 0.3595307 , 0.82296843, 0.31117923, 0.42294301]])

In [5]:
np.shape(train_a_y)

(400, 1)

In [6]:
train_a_y[:2, :]

array([[1],
       [1]])

Looks good.

### 2. Keras' Functional API Formalism

Keras' Functional API allows us to construct non-sequential models with multiple inputs, multiple outputs, branched models,  etc..

Key aspects:

* start with definition of inputs through Input layer(s)
* define the layers and outputs as:
$$\rm next\_layer\_activations = layer(layer\_parameters) (previous\_layer\_activations)$$
* define the model the the input(s) and output(s).  
* 'compile' the model by specifying loss function(s), metric(s) and the optimizer.  

We first set things up for one binary classification. It is most convenient to define a function called *build_model()* where we can parametrize aspects we want to vary.

In [7]:
def build_model(activation_function='relu', 
                optimizer='adam', 
                kernel_initializer=tf.keras.initializers.he_normal(seed=1), 
                bias_initializer='zeros'):

    # Define Input layer(s)
    input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

    # Define one hidden layer and act on input
    dense_layer = tf.keras.layers.Dense(10, 
                                        activation=activation_function, 
                                        kernel_initializer=kernel_initializer, 
                                        bias_initializer=bias_initializer,
                                        name='dense') # layer definition
    dense_output = dense_layer(input_numbers)  # layer acting on previous layer's output
    
    # Define one hidden layer and act on input
    dense_layer_2 = tf.keras.layers.Dense(10, 
                                        activation=activation_function, 
                                        kernel_initializer=kernel_initializer, 
                                        bias_initializer=bias_initializer,
                                        name='dense_2') # layer definition
    dense_output_2 = dense_layer_2(dense_output)  # layer acting on previous layer's output

    

    # Define classification layer and act on previous output to classify examples
    classification_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='classification') # layer definition
    classification_output = classification_layer(dense_output_2)   # layer acting on previous layer's output

    # Build and compile model
    model = tf.keras.models.Model(input_numbers, classification_output)
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    return model

model = build_model()


model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_numbers (InputLayer)   [(None, 10)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
classification (Dense)       (None, 1)                 11        
Total params: 231
Trainable params: 231
Non-trainable params: 0
_________________________________________________________________


Easy!   

**Question:** Do the dimensions look right? 

### 3.  Training Optimizations

Great, let's do some experimenting. We run the model above for the synthetic data set we defined, only considering the first label, and look at a few variations. Specifically, let's play with:

* the **activation function** in the hidden layer     
* the **initialization**    
* the **optimizer**

We run everything for 40 epochs and compare the results. (We split the training into two parts to cut down on reporting. 'verbose=0' supresses output. But we still want to see the final loss so we add one epoch with 'verbose=1'.)

In [8]:
model_1 = build_model()  # This one should be good...

model_1.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_1.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 88us/sample - loss: 0.2598 - val_loss: 0.2270


This is our supposed best setup, and it defines our baselines. 

Let's veer off and see how loss, etc. change. First, we use $tanh()$ as opposed to $relu()$:

In [9]:
model_2 = build_model(activation_function='tanh')  

model_2.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_2.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 90us/sample - loss: 0.2664 - val_loss: 0.2296


A bit worse. [Of course, the initialization is random, so things may be different on future runs.]

Next, we change the optimizer to stochastic gradient descent:

In [10]:
model_3 = build_model(optimizer='sgd')  

model_3.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_3.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 88us/sample - loss: 0.4956 - val_loss: 0.4756


A lot worse!!

Going back to the activation function, what about $sigmoid()$?

In [11]:
model_4 = build_model(activation_function='sigmoid')  

model_4.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_4.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 95us/sample - loss: 0.5621 - val_loss: 0.5476


Not good!

Lastly, what about choosing general random matrix initialization as opposed to He/Xavier? 

In [12]:
model_5 = build_model(kernel_initializer='random_normal')  

model_5.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_5.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 88us/sample - loss: 0.2325 - val_loss: 0.1902


A bit worse too.

**Critical Note:** We were cheating a bit here! There is a stochastic component and if one re-reruns the cells results will differ. In fact, *in this simple case* the last configuration is often the best.

### 4. Splitting Models - Multiple Outputs

Now, let's go back to our dataset and let's try to predict two labels. We want to model that by *splitting* the model, i.e., two distinct hidden layers act on the input, and we use those two branches to predict the two labels:  

In [13]:
# Define Input layer(s)
input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

# Define separate hidden layers and acting on (same!) input
dense_layer_1 = tf.keras.layers.Dense(10, activation='relu', name='dense_1')
dense_layer_2 = tf.keras.layers.Dense(10, activation='relu', name='dense_2')

dense_output_1 = dense_layer_1(input_numbers)     # hideen layer 1 acting on input
dense_output_2 = dense_layer_2(input_numbers)     # hideen layer 2 acting on input



# Define classification layer and act on previous output to classify examples
classification_layer_1 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_1')  # classification layer 1 for first branch

classification_layer_2 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_2')  # classification layer 2 for second branch


classification_output_1 = classification_layer_1(dense_output_1)
classification_output_2 = classification_layer_2(dense_output_2)



# Build and compile model

model_input = input_numbers
model_output = [classification_output_1, classification_output_2]
losses = ['binary_crossentropy', 'binary_crossentropy']
metrics = ['acc', 'acc']

model = tf.keras.models.Model(model_input, model_output)
model.compile(loss=losses,  optimizer='adam', metrics=metrics)




In [14]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_numbers (InputLayer)      [(None, 10)]         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10)           110         input_numbers[0][0]              
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 10)           110         input_numbers[0][0]              
__________________________________________________________________________________________________
classification_1 (Dense)        (None, 1)            11          dense_1[0][0]                    
____________________________________________________________________________________________

In [15]:
model.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=150,
    verbose=1
)

Train on 400 samples, validate on 400 samples
Epoch 1/150
400/400 [==============================] - 1s 2ms/sample - loss: 1.5062 - classification_1_loss: 0.8096 - classification_2_loss: 0.6986 - classification_1_acc: 0.5000 - classification_2_acc: 0.5525 - val_loss: 1.4823 - val_classification_1_loss: 0.7621 - val_classification_2_loss: 0.7076 - val_classification_1_acc: 0.5000 - val_classification_2_acc: 0.5475
Epoch 2/150
400/400 [==============================] - 0s 105us/sample - loss: 1.4565 - classification_1_loss: 0.7759 - classification_2_loss: 0.6774 - classification_1_acc: 0.5000 - classification_2_acc: 0.5500 - val_loss: 1.4425 - val_classification_1_loss: 0.7403 - val_classification_2_loss: 0.6941 - val_classification_1_acc: 0.5000 - val_classification_2_acc: 0.5375
Epoch 3/150
400/400 [==============================] - 0s 99us/sample - loss: 1.4221 - classification_1_loss: 0.7542 - classification_2_loss: 0.6662 - classification_1_acc: 0.5000 - classification_2_acc: 0.5750

Epoch 23/150
400/400 [==============================] - 0s 106us/sample - loss: 1.0889 - classification_1_loss: 0.4769 - classification_2_loss: 0.6086 - classification_1_acc: 0.8025 - classification_2_acc: 0.6925 - val_loss: 1.0760 - val_classification_1_loss: 0.4464 - val_classification_2_loss: 0.6393 - val_classification_1_acc: 0.8650 - val_classification_2_acc: 0.6400
Epoch 24/150
400/400 [==============================] - 0s 105us/sample - loss: 1.0756 - classification_1_loss: 0.4682 - classification_2_loss: 0.6078 - classification_1_acc: 0.8100 - classification_2_acc: 0.7000 - val_loss: 1.0618 - val_classification_1_loss: 0.4349 - val_classification_2_loss: 0.6366 - val_classification_1_acc: 0.8625 - val_classification_2_acc: 0.6375
Epoch 25/150
400/400 [==============================] - 0s 104us/sample - loss: 1.0628 - classification_1_loss: 0.4591 - classification_2_loss: 0.6022 - classification_1_acc: 0.8150 - classification_2_acc: 0.7000 - val_loss: 1.0488 - val_classification

Epoch 45/150
400/400 [==============================] - 0s 105us/sample - loss: 0.8916 - classification_1_loss: 0.3430 - classification_2_loss: 0.5504 - classification_1_acc: 0.8450 - classification_2_acc: 0.7125 - val_loss: 0.8785 - val_classification_1_loss: 0.3013 - val_classification_2_loss: 0.5865 - val_classification_1_acc: 0.9000 - val_classification_2_acc: 0.6850
Epoch 46/150
400/400 [==============================] - 0s 105us/sample - loss: 0.8850 - classification_1_loss: 0.3360 - classification_2_loss: 0.5505 - classification_1_acc: 0.8450 - classification_2_acc: 0.7100 - val_loss: 0.8732 - val_classification_1_loss: 0.2981 - val_classification_2_loss: 0.5844 - val_classification_1_acc: 0.9025 - val_classification_2_acc: 0.6850
Epoch 47/150
400/400 [==============================] - 0s 102us/sample - loss: 0.8799 - classification_1_loss: 0.3314 - classification_2_loss: 0.5446 - classification_1_acc: 0.8450 - classification_2_acc: 0.7100 - val_loss: 0.8677 - val_classification

Epoch 67/150
400/400 [==============================] - 0s 107us/sample - loss: 0.7902 - classification_1_loss: 0.2879 - classification_2_loss: 0.5094 - classification_1_acc: 0.8800 - classification_2_acc: 0.7550 - val_loss: 0.7801 - val_classification_1_loss: 0.2462 - val_classification_2_loss: 0.5410 - val_classification_1_acc: 0.9100 - val_classification_2_acc: 0.7200
Epoch 68/150
400/400 [==============================] - 0s 107us/sample - loss: 0.7869 - classification_1_loss: 0.2884 - classification_2_loss: 0.5038 - classification_1_acc: 0.8775 - classification_2_acc: 0.7575 - val_loss: 0.7771 - val_classification_1_loss: 0.2446 - val_classification_2_loss: 0.5390 - val_classification_1_acc: 0.9100 - val_classification_2_acc: 0.7200
Epoch 69/150
400/400 [==============================] - 0s 105us/sample - loss: 0.7832 - classification_1_loss: 0.2776 - classification_2_loss: 0.4996 - classification_1_acc: 0.8825 - classification_2_acc: 0.7575 - val_loss: 0.7739 - val_classification

Epoch 89/150
400/400 [==============================] - 0s 103us/sample - loss: 0.7294 - classification_1_loss: 0.2560 - classification_2_loss: 0.4715 - classification_1_acc: 0.8850 - classification_2_acc: 0.7825 - val_loss: 0.7228 - val_classification_1_loss: 0.2163 - val_classification_2_loss: 0.5099 - val_classification_1_acc: 0.9125 - val_classification_2_acc: 0.7500
Epoch 90/150
400/400 [==============================] - 0s 100us/sample - loss: 0.7272 - classification_1_loss: 0.2613 - classification_2_loss: 0.4706 - classification_1_acc: 0.8875 - classification_2_acc: 0.7775 - val_loss: 0.7214 - val_classification_1_loss: 0.2163 - val_classification_2_loss: 0.5088 - val_classification_1_acc: 0.9175 - val_classification_2_acc: 0.7525
Epoch 91/150
400/400 [==============================] - 0s 100us/sample - loss: 0.7250 - classification_1_loss: 0.2538 - classification_2_loss: 0.4730 - classification_1_acc: 0.8875 - classification_2_acc: 0.7775 - val_loss: 0.7187 - val_classification

Epoch 111/150
400/400 [==============================] - 0s 102us/sample - loss: 0.6895 - classification_1_loss: 0.2351 - classification_2_loss: 0.4561 - classification_1_acc: 0.8925 - classification_2_acc: 0.8050 - val_loss: 0.6865 - val_classification_1_loss: 0.1970 - val_classification_2_loss: 0.4900 - val_classification_1_acc: 0.9200 - val_classification_2_acc: 0.7600
Epoch 112/150
400/400 [==============================] - 0s 102us/sample - loss: 0.6883 - classification_1_loss: 0.2411 - classification_2_loss: 0.4533 - classification_1_acc: 0.8975 - classification_2_acc: 0.8000 - val_loss: 0.6851 - val_classification_1_loss: 0.1960 - val_classification_2_loss: 0.4894 - val_classification_1_acc: 0.9150 - val_classification_2_acc: 0.7600
Epoch 113/150
400/400 [==============================] - 0s 101us/sample - loss: 0.6864 - classification_1_loss: 0.2318 - classification_2_loss: 0.4476 - classification_1_acc: 0.8950 - classification_2_acc: 0.8000 - val_loss: 0.6839 - val_classificat

Epoch 133/150
400/400 [==============================] - 0s 104us/sample - loss: 0.6619 - classification_1_loss: 0.2189 - classification_2_loss: 0.4349 - classification_1_acc: 0.9075 - classification_2_acc: 0.8100 - val_loss: 0.6640 - val_classification_1_loss: 0.1841 - val_classification_2_loss: 0.4785 - val_classification_1_acc: 0.9150 - val_classification_2_acc: 0.7675
Epoch 134/150
400/400 [==============================] - 0s 104us/sample - loss: 0.6610 - classification_1_loss: 0.2181 - classification_2_loss: 0.4396 - classification_1_acc: 0.9075 - classification_2_acc: 0.8100 - val_loss: 0.6630 - val_classification_1_loss: 0.1837 - val_classification_2_loss: 0.4780 - val_classification_1_acc: 0.9175 - val_classification_2_acc: 0.7675
Epoch 135/150
400/400 [==============================] - 0s 105us/sample - loss: 0.6599 - classification_1_loss: 0.2174 - classification_2_loss: 0.4344 - classification_1_acc: 0.9125 - classification_2_acc: 0.8125 - val_loss: 0.6630 - val_classificat

**Questions:**
* Does it surprise you that the loss for label 1 is smaller than the loss for label 2?    
* Quick side check... do we overtrain?    

### 5. Custom Layers

Let's define a simple custom layer which is nothing but an affine transformation followed by a relu activation. We need to define:
* \_\_init\_\_()      
* build()   
* call()   
* compute_output_shape()   

In [16]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer

class MyStupidDenseLayer(Layer):

    def __init__(self, output_dim, **kwargs):   
        self.output_dim = output_dim
        
        super(MyStupidDenseLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.W = self.add_variable("W",
                                    shape=[int(input_shape[1]),self.output_dim], 
                                   initializer=tf.keras.initializers.GlorotNormal(),
                                   trainable=True)
        self.b = self.add_variable("b",
                                    shape=[self.output_dim, ], 
                                   initializer=tf.initializers.zeros(),
                                   trainable=True)
        super(MyStupidDenseLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return tf.nn.relu(tf.tensordot(x, self.W, axes=[[1],[0]]) + self.b)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

Let's use our custom layer in the same architecture. 

In [17]:
# Define Input layer(s)
input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

# Define separate hidden layers and acting on (same!) input
dense_layer_1 = MyStupidDenseLayer(10, name='dense_1')
dense_layer_2 = MyStupidDenseLayer(10, name='dense_2')

dense_output_1 = dense_layer_1(input_numbers)
dense_output_2 = dense_layer_2(input_numbers)



# Define classification layer and act on previous output to classify examples
classification_layer_1 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_1')

classification_layer_2 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_2')


classification_output_1 = classification_layer_1(dense_output_1)
classification_output_2 = classification_layer_2(dense_output_2)



# Build and compile model

model_input = input_numbers
model_output = [classification_output_1, classification_output_2]
losses = ['binary_crossentropy', 'binary_crossentropy']
metrics = ['acc', 'acc']

model = tf.keras.models.Model(model_input, model_output)
model.compile(loss=losses,  optimizer='adam', metrics=metrics)




Instructions for updating:
Please use `layer.add_weight` method instead.


In [18]:
model.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=149,
    verbose=0
)

model.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=1,
    verbose=1
)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 110us/sample - loss: 0.6301 - classification_1_loss: 0.2007 - classification_2_loss: 0.4250 - classification_1_acc: 0.9325 - classification_2_acc: 0.8100 - val_loss: 0.6326 - val_classification_1_loss: 0.1679 - val_classification_2_loss: 0.4642 - val_classification_1_acc: 0.9225 - val_classification_2_acc: 0.7850


Close enough. So the custom layer works exactly as expected.


### 6. Sharing & Freezing Weights 


Let's define an identical model. The weights for this model will be re-initialized, so upon training for 1 epoch the loss will be much higher than after 150 epochs for the original model: 

In [19]:
# Define Input layer(s)
input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

# Define separate hidden layers and acting on (same!) input
dense_layer_1 = MyStupidDenseLayer(10, name='dense_1')
dense_layer_2 = MyStupidDenseLayer(10, name='dense_2')

dense_output_1 = dense_layer_1(input_numbers)
dense_output_2 = dense_layer_2(input_numbers)



# Define classification layer and act on previous output to classify examples
classification_layer_1 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_1')

classification_layer_2 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_2')


classification_output_1 = classification_layer_1(dense_output_1)
classification_output_2 = classification_layer_2(dense_output_2)



# Build and compile model

model_input = input_numbers
model_output = [classification_output_1, classification_output_2]
losses = ['binary_crossentropy', 'binary_crossentropy']
metrics = ['acc', 'acc']

model_mirror = tf.keras.models.Model(model_input, model_output)
model_mirror.compile(loss=losses,  optimizer='adam', metrics=metrics)




In [20]:
model_mirror.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=1,
    verbose=1
)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 1s 2ms/sample - loss: 1.5032 - classification_1_loss: 0.8021 - classification_2_loss: 0.7035 - classification_1_acc: 0.5000 - classification_2_acc: 0.4725 - val_loss: 1.4658 - val_classification_1_loss: 0.7578 - val_classification_2_loss: 0.6975 - val_classification_1_acc: 0.5000 - val_classification_2_acc: 0.4925


Indeed, this model would make very different predictions - the loss is ~1.4 vs ~0.6. No surprise, obviously, as this model has not been trained.

What if we manually set the the weights of the second model to the weights of the first? Let's look at the layers for both models and then copy the weights:

In [21]:
model.layers

Let's look at one layer in more detail:

In [22]:
len(model.layers[1].get_weights())

2

Why would that be? No magic.. weights and biases of layer!

In [23]:
np.shape(model.layers[1].get_weights()[0])

(10, 10)

In [24]:
np.shape(model.layers[1].get_weights()[1])

(10,)

Cool. What is the name of the layer?

In [25]:
model.layers[1].name

'dense_1'

Correct. Is it trainable?

In [26]:
model.layers[1].trainable

True

Great. Are the layers of the second model essentially the same?

In [27]:
model_mirror.layers

Yup. (We could look at shapes, etc, but won't do that now.) Now we set the second model's weights to equal the first model's trained weights:

In [28]:
model_mirror.layers[1].set_weights(model.layers[1].get_weights())
model_mirror.layers[2].set_weights(model.layers[2].get_weights())
model_mirror.layers[3].set_weights(model.layers[3].get_weights())
model_mirror.layers[4].set_weights(model.layers[4].get_weights())

**Question:** Why don't we set the weights for layer 0?

Okay. What is the loss now for the mirror model? 

In [29]:
model_mirror.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=1,
    verbose=1
)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 110us/sample - loss: 0.6311 - classification_1_loss: 0.2038 - classification_2_loss: 0.4212 - classification_1_acc: 0.9250 - classification_2_acc: 0.8125 - val_loss: 0.6334 - val_classification_1_loss: 0.1688 - val_classification_2_loss: 0.4639 - val_classification_1_acc: 0.9250 - val_classification_2_acc: 0.7850


Cool! Essentially the same as for the original one! (Not a surprise actually... but great that it works.)

Let's now turn to **freezing** layers. For example, you may want to copy weights from another model but hold those weights fixed upon further training. To test this, we will do a simple toy exercise: freeze a layer of the second model and train it further. Let's compare some weights before and after training. 

We start by defining a new model that again has the same architecture, **but we set the trainable-parameter for one layer to False**: 

In [30]:
# Define Input layer(s)
input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

# Define separate hidden layers and acting on (same!) input
dense_layer_1 = MyStupidDenseLayer(10, name='dense_1')

dense_layer_2 = MyStupidDenseLayer(10, name='dense_2')
dense_layer_2.trainable = False                                   # Freeze this layer

dense_output_1 = dense_layer_1(input_numbers)
dense_output_2 = dense_layer_2(input_numbers)



# Define classification layer and act on previous output to classify examples
classification_layer_1 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_1')

classification_layer_2 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_2')



classification_output_1 = classification_layer_1(dense_output_1)
classification_output_2 = classification_layer_2(dense_output_2)



# Build and compile model

model_input = input_numbers
model_output = [classification_output_1, classification_output_2]
losses = ['binary_crossentropy', 'binary_crossentropy']
metrics = ['acc', 'acc']

model_mirror_2 = tf.keras.models.Model(model_input, model_output)
model_mirror_2.compile(loss=losses,  optimizer='adam', metrics=metrics)


Next, we again set the weights to mirror the first model:

In [31]:
model_mirror_2.layers[1].set_weights(model.layers[1].get_weights())
model_mirror_2.layers[2].set_weights(model.layers[2].get_weights())
model_mirror_2.layers[3].set_weights(model.layers[3].get_weights())
model_mirror_2.layers[4].set_weights(model.layers[4].get_weights())

What are (some of) the weights now, before we train further?

In [32]:
model_mirror_2.layers[1].get_weights()[0][:3, :3]

array([[ 0.16306525,  0.3430157 , -0.13253239],
       [-0.1442931 ,  0.7501852 , -0.06985269],
       [-0.4240862 ,  0.58976454, -0.3947481 ]], dtype=float32)

In [33]:
model_mirror_2.layers[2].get_weights()[0][:3, :3]

array([[ 0.58394676,  0.05053842, -0.35226533],
       [-0.01004035,  0.3307187 , -0.03672026],
       [-0.2914849 , -0.14890574, -0.67025244]], dtype=float32)

We will compare that with the values post further training. Let's also check the 'trainable' settings:

In [34]:
model_mirror_2.layers[1].trainable

True

In [35]:
model_mirror_2.layers[2].trainable

False

Good, so it shows that one hidden layer supposedly is trainable, the other one is not. Is that true? We train more and compare weights compared to what they were:

In [36]:
model_mirror_2.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=40,
    verbose=0
)
model_mirror_2.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=1,
    verbose=1
)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 110us/sample - loss: 0.6161 - classification_1_loss: 0.1892 - classification_2_loss: 0.4205 - classification_1_acc: 0.9275 - classification_2_acc: 0.8125 - val_loss: 0.6260 - val_classification_1_loss: 0.1608 - val_classification_2_loss: 0.4639 - val_classification_1_acc: 0.9275 - val_classification_2_acc: 0.7800


In [37]:
model_mirror_2.layers[1].get_weights()[0][:3, :3]

array([[ 0.20854098,  0.35762998, -0.13515098],
       [-0.16344354,  0.7268433 , -0.046384  ],
       [-0.4516486 ,  0.61464345, -0.4081515 ]], dtype=float32)

These weights **did change**.

In [38]:
model_mirror_2.layers[2].get_weights()[0][:3, :3]

array([[ 0.58394676,  0.05053842, -0.35226533],
       [-0.01004035,  0.3307187 , -0.03672026],
       [-0.2914849 , -0.14890574, -0.67025244]], dtype=float32)

These weights **did not change**. So here we could see layer freezing at work!